# ChannelAttentionFusionT — Hata Avı Çalışmaları (Soru–Çözüm Derlemesi)

Bu not defteri, **ChannelAttentionFusionT** (CA) bloğu üzerinde yapılan “tek satır değişiklik” senaryolarını,
bu değişikliklerin **davranışsal etkilerini**, **beklenen eğitim belirtilerini** ve **en hızlı debug kontrollerini**
toplu biçimde sunar.

> Odak: *sample-wise adaptiflik*, *stabil öğrenme*, *mask/temperature davranışı*, *router doğruluğu*.


## Referans: Doğru (beklenen) akış

- `avg_s = AvgPool(x)`, `max_s = MaxPool(x)` → (B,C,1,1)  
- `a = MLP(avg_s)`, `m = MLP(max_s)` → (B,C,1,1)  
- `fusion_router([avg_s,max_s])` → logits (B,2)  
- `fusion_w = softmax(logits, dim=1)` → (B,2) **(her örnek kendi içinde normalize)**  
- `z = w0*a + w1*m`  
- `ca = gate(z / T)`  
- `y = x * ca`  

**Kritik invariant:** `fusion_w.sum(dim=1) ≈ 1` (her örnek için).


## Senaryo 1: Softmax ekseni hatası (dim=0)

### Yapılan değişiklik
```python
fusion_w = torch.softmax(logits, dim=0)
```

### Neden sorunlu?
- `logits` şekli (B,2). `dim=0` seçilirse normalizasyon **batch boyunca** yapılır.
- Sonuç: ağırlıklar örnek içi (avg vs max) yarışmak yerine örnekler birbirini etkiler.
- Bu durum **sample-wise adaptifliği bozar** ve batch kompozisyonuna bağımlılık üretir.

### Eğitimde beklenen belirtiler
- Aynı örnek farklı batch’te farklı `fusion_w` alabilir.
- Batch size/kompozisyon değişiminde performans dalgalanması.
- Daha gürültülü öğrenme; genelleme düşebilir.

### En hızlı debug kontrolleri
- Doğru durumda: `fusion_w.sum(dim=1)` → (B,) her eleman ≈ 1 olmalı.
- Hatalı dim=0 imzası: `fusion_w.sum(dim=0)` → (2,) ≈ [1,1].
- Ek kontrol: `fusion_w.std(dim=0)` davranış anomalisini gösterebilir.

### Doğrusu (fix)
```python
fusion_w = torch.softmax(logits, dim=1)
```


## Senaryo 2: Temperature kullanımının ters çevrilmesi (z * T)

### Yapılan değişiklik
```python
ca = gate_fn(z * T)
```

### Neden sorunlu?
- Temperature, gate öncesi logitleri **bölerek** yumuşatma/sertleştirme amacı taşır.
- Çarpma yapılınca kontrol tersine döner: `T` büyüdükçe logit büyür → maske daha çok saturate olur.
- Özellikle `learnable_temperature=True` iken model T’yi büyütüp maskeyi kilitleyebilir.

### Eğitimde beklenen belirtiler
- `ca` hızla 0/1’e yapışabilir (saturasyon).
- Gradyan zayıflayabilir (sigmoid/hardsigmoid uç bölgelerde türev küçülür).
- Eğitim kararsızlaşabilir ya da erken “kilitlenme” görülebilir.

### En hızlı debug kontrolleri
- `T` arttıkça `ca` dağılımının daha keskinleşip keskinleşmediğini izleyin.
- `ca` istatistikleri: `min/max/mean/std` ve `P(ca>0.99)`/`P(ca<0.01)`.
- `|z/T|` yerine `|z*T|` büyümesi saturasyon işaretidir.

### Doğrusu (fix)
```python
ca = gate_fn(z / T)
```


## Senaryo 3: z’yi ham squeeze’tan üretmek (avg_s/max_s) — ifade gücü kaybı

### Yapılan değişiklik
```python
z = w0*avg_s + w1*max_s  # a,m yerine
```

### Neden sorunlu?
- `a` ve `m`, MLP ile **öğrenilmiş kanal dönüşümü** (C→hidden→C) içerir.
- `avg_s/max_s` ise yalnızca ham istatistik. `z` ham squeeze ile üretilirse CA’nın temsil gücü zayıflar.
- Kod çalışır; ancak attention katkısı azalır.

### Eğitimde beklenen belirtiler
- Konverjans yavaşlayabilir.
- `ca` daha az ayrıştırıcı olabilir; bazen “var ama etkisi az” hissi.
- Tavan performans düşebilir (görev/veri setine bağlı).

### En hızlı debug kontrolleri
- `ca`’nın kanallar arası varyansını izleyin.
- `a` ile `avg_s` arasındaki fark: `||a-avg_s||` (MLP’nin dönüşüm gücü).
- Ablation: a/m ile üretim vs avg_s/max_s ile üretim kıyası.

### Doğrusu (fix)
```python
z = w0*a + w1*m
```

**Not:** Bu bir “hata”dan çok tasarım zayıflatmasıdır; hedef güçlendirme ise `a/m` tercih edilir.


## Senaryo 4: MLP aktivasyonunun kaldırılması (lineerleşme)

### Yapılan değişiklik
```python
return fc2(fc1(s))  # act yok
```

### Neden sorunlu?
- Aktivasyon kaldırılınca iki lineer katman ardışık gelir ve tek lineer dönüşüme indirgenir.
- Nonlinearity kaybolur; kanal ilişkilerini modelleme kapasitesi düşer.

### Eğitimde beklenen belirtiler
- CA etkisi zayıflayabilir; maske daha “düz” davranabilir.
- Konverjans bazen yavaşlar; tavan performans düşebilir.
- Model yine öğrenir (backbone taşır) ama attention katkısı azalır.

### En hızlı debug kontrolleri
- `ca` ayrıştırıcılığı: std/variance, uçlara yığılma.
- `a/m` istatistikleri (std/mean) ve epoch boyunca değişim.
- Ablation: aktivasyonlu vs aktivasyonsuz kıyas.

### Doğrusu (fix)
```python
return fc2(act(fc1(s)))
```


## Senaryo 5: `get_T()` içinde +eps kaldırılması (T → 0 riski)

### Yapılan değişiklik
```python
return F.softplus(self.t_raw)  # +eps yok
```

### Neden sorunlu?
- Softplus pozitif üretse de çok küçük değerlere yaklaşabilir.
- `z/T` ifadesinde `T` çok küçülürse logit büyür → saturasyon/taşma riski.
- `eps` T’ye alt sınır koyarak sayısal stabilite sağlar.

### Eğitimde beklenen belirtiler
- `T` çok küçülürse maske keskinleşir; öğrenme kilitlenebilir.
- Aşırı durumda `z/T` büyür → inf/NaN riskleri.
- AMP/mixed precision ortamında risk artar.

### En hızlı debug kontrolleri
- `T.min()` kırmızı bayrak (örn <1e-4 gibi).
- `torch.isfinite(z/T)` ve `torch.isfinite(loss)` kontrolleri.
- `ca` saturasyon oranı artıyor mu?

### Doğrusu (fix)
```python
return F.softplus(self.t_raw) + self.eps
```


## Senaryo 6: Batch ortalaması + expand ile ‘global’ fusion_w (sample-wise ölür)

### Yapılan değişiklik
```python
s_cat = cat([avg_s,max_s], dim=1).mean(dim=0, keepdim=True)
logits = fusion_router(s_cat)  # (1,2)
fusion_w = softmax(logits, dim=1).expand(B,-1)  # tüm batch aynı
```

### Neden sorunlu?
- `mean(dim=0)` ile batch tek ‘ortalama örneğe’ indirgenir.
- `expand` ile aynı ağırlık tüm örneklere uygulanır.
- `fusion_w` şeklen (B,2) olsa da içerik olarak **her satır aynıdır** → sample-wise adaptiflik yok olur.

### Eğitimde beklenen belirtiler
- Batch kompozisyonu değiştikçe fusion_w değişir.
- Aynı örnek farklı batch’te farklı ağırlık alabilir.
- Genelleme ve stabilite olumsuz etkilenebilir.

### En hızlı debug kontrolleri
- Batch içi satır eşitliği:
  - `(fusion_w - fusion_w[0:1]).abs().max()` ≈ 0 ise şüpheli.
  - `fusion_w.std(dim=0)` ≈ 0 ise tüm örnekler aynı ağırlığı alıyordur.
- Aynı sample farklı batch’lerde: fusion_w değişiyor mu?

### Doğrusu (fix)
```python
s_cat = torch.cat([avg_s, max_s], dim=1)
logits = fusion_router(s_cat).flatten(1)
fusion_w = softmax(logits, dim=1)
```


## Senaryo 7: Detach ile CA’yı ‘öğrenemez’ hale getirmek (grad akışı kesilir)

### Yapılan değişiklik
```python
ca = gate_fn((z / T).detach())
```

### Neden sorunlu?
- `detach()` hesaplama grafiğinden koparır → `ca` üzerinden `z` ve `T` tarafına gradyan akmaz.
- Sonuç: fusion_router, MLP (fc1/fc2) ve learnable T (t_raw) attention üzerinden öğrenmeyi kaybeder.
- Model yine öğrenebilir (backbone), fakat CA bloğu ‘var ama öğrenmeyen’ hale gelir.

### Eğitimde beklenen belirtiler
- CA parametrelerinin grad’leri yok/çok küçük olabilir.
- CA maskesi epoch boyunca anlamlı değişmeyebilir.
- Performans artışı beklenen seviyeye ulaşmayabilir.

### En hızlı debug kontrolleri
- Gradient kontrolü:
  - `fc1.weight.grad`, `fc2.weight.grad`, `fusion_router[*].weight.grad`, `t_raw.grad`.
- Ablation: detach’li vs detach’siz performans.
- `ca` dağılımının zamanla değişimi.

### Doğrusu (fix)
```python
ca = gate_fn(z / T)
```

**Not:** Detach, teacher-student/target network gibi yapılarda bilinçli kullanılır; attention maskesi öğrenmesi isteniyorsa genelde kullanılmaz.


## Mini Kontrol Listesi (CA)

1) **Softmax ekseni**  
- `fusion_w` (B,2) → `sum(dim=1) ≈ 1`

2) **Temperature mantığı**  
- Doğru: `z / T` (T↑ → yumuşar)  
- Yanlış: `z * T` (T↑ → sertleşir)

3) **Learnable T stabilitesi**  
- `T.min()` çok küçükse (örn. `< 1e-4`) risk.

4) **MLP nonlinearity**  
- Aktivasyon yoksa lineerleşir; ayrıştırıcılık düşebilir.

5) **Sample-wise adaptiflik**  
- `fusion_w` satırları aynıysa (`std≈0`) sample-wise ölür.

6) **Gradient akışı**  
- `detach` varsa CA parametreleri grad alamaz.
